In [15]:
push!(LOAD_PATH, "./")
using BdgSolver

In [ ]:
workspace()
reload("BdgSolver")
using BdgSolver

In [ ]:
sh = Shape(10, 10, 10)
mat = Material("Sn", 1.0, 1.0, 1.0)
pars = Parameters(mat, sh)
sys = System(mat, sh, pars)
H = Hamiltonian(mat, sh, pars);

## Integration

Clearly, integrating like this won't be compatible with being able to apply corrections as convolutions to the thermal weight or DOS. I simply can't do "symbolic" integrations for all the corrections and get a function as end result. I'll have to discretize at least part of it, on which I can apply the corrections. I think it's cleanest if I just apply *all* corrections to the DOS:

$$M_{ij} = \Phi_{ij} \int_{DW} d\xi\, N_i(\xi) \int d\Xi\, F(\xi - \Xi, \Gamma_\xi) \frac{ \textrm{tanh} \frac{\beta_c \Xi}{2}}{\Xi}$$

If we put in the Debye-window cutoff manually as a characteristic function, $\chi_{DW}(\xi)$, both integrations become completely equivalent (at least in principle).

$$M_{ij} = \Phi_{ij} \int d\xi\, \chi_{DW}(\xi)N_i(\xi) \int d\Xi\, F(\xi - \Xi, \Gamma_\xi) \frac{ \textrm{tanh} \frac{\beta_c \Xi}{2}}{\Xi}.$$

Reversing the integrations:
$$M_{ij} = \Phi_{ij} \int d\Xi\, \frac{ \textrm{tanh} \frac{\beta_c \Xi}{2}}{\Xi}  \left\{\int d\xi\, F(\xi - \Xi, \Gamma_\xi) \chi_{DW}(\xi)N_i(\xi)\right\}.$$

This way, I can repeatedly keep applying corrections to the DOS. I think. Right?


Note that, if I choose to discretize the DOS, and apply all corrections at the DOS level, I will essentially be multiplying the number of operations (and memory usage) by a factor $\nu$... 

## Thermal determinant

Thought of something!

The determinant I have to compute essentialy comes down to $\det\left(\Phi_{ij} I_i\right)$, where $\Phi_{ij}$ are the overlap elements, and $I_i$ is the thermal integral.

We can easily write this down as a proper matrix product: $\Phi_{ij} (\delta_{jk}I_k)$. That is to say, the same can be achieved by multiplying $\Phi$ with a matrix $I$ having the $I_i$ on the diagonal. The determinant of this matrix product factorizes, of course, $\det(\Phi I) = \det\Phi \det I$, and we are left to compute the determinant of $\Phi$ (trivial), and the determinant of the diagonal matrix $I$, which is cleary very easy.

I wonder if there's a difference in computation time of calculating the determinant simply as the product, or running the `det` function.

In [248]:
x = rand(10000) + 0.6
@time X = diagm(x)
@time prod(x)
@time det(X);

 13.470774 seconds (7 allocations: 762.940 MB, 0.49% gc time)
  0.000024 seconds (5 allocations: 176 bytes)
  0.148644 seconds (8 allocations: 78.375 KB)


Ah yes, not only is the determinant a sot more work (by a factor 1000 or so), just STORING the diagonal matrix is ridiculous. Of course, I could simply store it as a sparse matrix, but then I would get the overhead of going to and from a sparse form, right?

This could make stuff a lot more efficient!

Okay, granted, $\Phi$ is only a $\nu\times\nu$ matrix, so it's not terribly expensive, but still, this should help.

In [9]:
BdgSolver.get_Tc(sys)

ξs shape: (50,)


0.043199007720004505

Weight shape: (50,)
N shape: (50,11)
I shape: (1,11)


Next challenge: The determinant will, because of the windowing, inherently contain a whole bunch of 